# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../WeatherPy/cities.csv"
vacation_df = pd.read_csv(csv_path)
vacation_df.head()
# vacation_df.info()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mirny,62.5353,113.9611,-26.50,97,100,1.09,RU,1645069569
1,1,Geraldton,-28.7667,114.6000,40.23,10,0,10.29,AU,1645069569
2,2,Cabo San Lucas,22.8909,-109.9124,21.61,63,33,5.65,MX,1645069319
3,3,Cayenne,4.9333,-52.3333,26.16,84,75,1.54,GF,1645069547
4,4,Nang Rong,14.6377,102.7914,28.20,62,60,3.24,TH,1645069570


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Reference - Class 3 - 07-Stu_Airport_Map

# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = vacation_df[["Lat", "Lng"]]

# Convert from int to float
humidity = vacation_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=5,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Conditions:
#  A max temperature lower than 40 degrees but higher than -33.
#  Wind speed less than 20 mph.
#  Zero cloudiness.
# Reference: https://keytodatascience.com/selecting-rows-conditions-pandas-dataframe/
     
weather_criteria_df = pd.DataFrame(vacation_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

weather_con_df = weather_criteria_df[(weather_criteria_df["Max Temp"] > -33) & (weather_criteria_df["Max Temp"] < 40) & (weather_criteria_df["Wind Speed"] < 20) & (weather_criteria_df["Cloudiness"] > 0)]
weather_con_df.head()


,City,Max Temp,Wind Speed,Cloudiness
0,Mirny,-26.50,1.09,100
2,Cabo San Lucas,21.61,5.65,33
3,Cayenne,26.16,1.54,75
4,Nang Rong,28.20,3.24,60
5,New Norfolk,24.40,3.13,2


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Dataframe
hotel_df = pd.DataFrame(vacation_df, columns = ["City", "Country", "Lat", "Lng"])

# Set up additional column to hold hotel information
hotel_df["Hotel Name"] = ""

hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Mirny,RU,62.5353,113.9611,
1,Geraldton,AU,-28.7667,114.6000,
2,Cabo San Lucas,MX,22.8909,-109.9124,
3,Cayenne,GF,4.9333,-52.3333,
4,Nang Rong,TH,14.6377,102.7914,


In [16]:
# Reference - Code is from Day 3 class - 05-Stu_Google_Complex - 01:59:39

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key
}


# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_hotel = requests.get(base_url, params=params)

    # convert to json
    name_hotel = name_hotel.json()
    print(json.dumps(name_hotel, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_hotel["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "next_page_token": "Aap_uED8H5Uvo7UpdZXSIiquEddTOHr_-HRFbiT1txT0fVC8opTGtVHu0S63o94mnGUjizOtyy9HfV1VS38SVtRSP7mvcIvOGrhC5vnN1u3okiFDbLtESkSUxMv541IdOC11_AHTsI_0z-hn09y3Wa4S5XMPu2cf2yYSpznsOFX5w664i4k9Tk1JLEpkEMee6iVVQJCeI8Qqe81QMaT2j82KD0MC_yeIxEcB0Td0j-MnFlMvd8iuGMJB0CeChBUmvdCW-Pk4VNE1nBs8F29ynYZAvpsbvpqtp7W3JvNuErdJmJlbCtRAp-eQoBzUf58OYgHw1HZquM1Jv4Q9K7FSVRUefSih_ihhVEUzJgHPJcncJQTJfC7qQQ002BHjjxa6HEB8z7AuiowZGOWzU8PHQ8H9vbIumtChmGau2iZ9J_Os34Dmtr2yeCFtSg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 62.5313374,
                    "lng": 113.9773881
                },
                "viewport": {
                    "northeast": {
                        "lat": 62.56367604612313,
                        "lng": 114.044519006685
                    },
                    "southwest": {
                        "lat": 62.50793897525757,
                        "lng": 113.9343518420336


{
    "html_attributions": [],
    "next_page_token": "Aap_uED3ptE4XMEcfK04Oy79vtHEWgRXSlsyLyQ5m1aDMkR50wAid844I5qu_ha5o5e60tYmwBIU5D0EuQrNTNN-LJkskj6Ssvlo1Oz6k719cbXNqhfjto7iOTagUvxRiRD608OQS70KPc6zCi1VfkZaBAJteSXxeyskGH0p26Z6mHi-h9odwopJHTQYXgiSDhnZSvML5TuzA4zCqxnIKHhMc7KK8e2afzHadQLsMBKpl1AxaKDCzF1AgTbK1C89885890mFuxgE156jWiJt9isV_XgNXOFAER7czfx7KFAZL2qDb6PruVenLAWFATFw2AI7g_U0fmqNarvOiQOU09tYKTI5D8cfCiJJuq6VclUidmsPqM319_qLwV9NCMI4_YST7O05g8mnURc_InCWunJw208xgj-M4AjF611cwJMxD-MzXMR3XzJxxw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -28.7761346,
                    "lng": 114.611295
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.76320798509792,
                        "lng": 114.6261427696306
                    },
                    "southwest": {
                        "lat": -28.7886324981025,
                        "lng": 114.596259950611

{
    "html_attributions": [],
    "next_page_token": "Aap_uECQgDssfpi6ampgYs9fxXDaeMphCPN68F9z96JTWps7KilHElybsEVAWnFMDpnIvbEo-nfkkohk2mxmg9YwsNOVk_svFAXcLrPHw-VxJReH-s149Tt5dA-71e584kWE1NuJJ7QOEWvT60n17bgNUIf2kiGJn8FCjVetpJNrax-Z0OL7PiTbWmNXNgNLqsT33K6ar1mmg6n7iP56ursQheQxBFjB6ehv5rB1mhywBGM3Gj3c1iouG_z2SezkqeYuOlh5tszV43qCpytdpfIOUhYcyuCf46hJPD6g6y0uIHXKM2rA5-ABm1VJechtbtIRlQqkqrZgxkoSrCrPu3H5nKNP0mou-Lx3JrEP6u4elEeeZFllvgMpV6YpODYmjEiJVYWkrxL2baQqkuoI1bTM2tSDT2Zu-51fW9mWlqtBLrM2-BoBhajOOw",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 22.8905327,
                    "lng": -109.9167371
                },
                "viewport": {
                    "northeast": {
                        "lat": 22.92028757787479,
                        "lng": -109.8942558770843
                    },
                    "southwest": {
                        "lat": 22.87216499597992,
                        "lng": -109.9717402828

{
    "html_attributions": [],
    "next_page_token": "Aap_uED68WuBxaeaq3E1r2WoLNuHVWYjKtJ80iK_mw2Oo3E4LQtX4gfnoPWnaCHqiI8Awd9BJYZanmtc9rUICpID2MzVBA2wHYuoiTKrBoqcb9gGqKv20WMCduCdiJ93TjjR4hsQcCZNwjwGvrOU-JUlt6Y9yx_EVUC9C_dd5B4jOmkNPdiNaGXRu-jpjwyHSCGFq9LYDKyBp5Is0x9FK0Xc3X759eQCOG8easGLpcYm93w49cXgMNzvMETNXfnDUcQ-D63Ymks4DcnSry6_WgjRH4sviVX_pX5Su7kq0lNwSUZwa-DGmLjQjB8GMZvp3HQOMhIYox_ZcWMTEXndT2GzLUOcarBoui23ST4j6_blwf2fKDLl6sRH9G3Kd8qZ2TC861K529KvUlvkYqk6iKgI5Y_7hjEmLc1cXIYZgRMV7ZpvXRqm65zSGg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 4.92242,
                    "lng": -52.3134531
                },
                "viewport": {
                    "northeast": {
                        "lat": 4.951705390698261,
                        "lng": -52.2801374948585
                    },
                    "southwest": {
                        "lat": 4.88767638044199,
                        "lng": -52.34268668281687
  

{
    "html_attributions": [],
    "next_page_token": "Aap_uEA977nq8ROpI8wBlh_xvANOl8VPkmnsiLovAo_n4cOjWQ99aqoX0ddM0l8NmBD1-AAmyS0NzEaM2B3T1Sv-x2t5_ZswIr7wtvk6HjQcALoUQAyVCf6tDh1htHNjgD3YtyO1b9DfAVNZMjIt14iSX9gJNrulHwSMQ2VNvESMhyDS2JElnQuL2Dpwru3JrPuzSzNFzvS4D8eGkKrOiUa_rd9fs7RCZzYn6XvKTqrkosqC5wpQstFtMuhqDJyObpOuYOuTKerMpruHAwMhJTJ847EuC1UtBUP91f40D5ZG0F8fgAfR5hUmIaAP0KlGQ6N2mNexIqMdliuFbV72YpSHXHQ7MkfVoQPYa3TddTrZ_zgCJP5nzXHdwS27GRj5ixjLbeAuZvZInvz3LRBZt_F0X1zQLF9AaJEiaVVcZElukZVTwbcideA-5A",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": 14.6335733,
                    "lng": 102.7934934
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.71497882256561,
                        "lng": 102.8282298502953
                    },
                    "southwest": {
                        "lat": 14.59970953709549,
                        "lng": 102.742863465447


{
    "html_attributions": [],
    "next_page_token": "Aap_uECwT18M8v9iXs6NSiKWV5W-zQ94OlhjcD60lzWTLDz7x-oLvXLW8p5OMkTTQQVYWn9aXWlHJ726-vqRA0OHZ0xQ_WQM1POGEe0CRsMM5jDr7yw_5ah2UvgMShCSAObnM_sZlLnRAfFJJn_PU26upEcPnBzc-0W5VBvLOnnnciqDqkyBKkYxvwergt-N8OHRSgCYKY5f_yshtq3MI9iQDgPkVrzLJpSqUQ-ccYFMP0nEZa4TUPt7LNCP1qgL1TQVaXuw7GrYpjfoU8aKYASZuCnOoCo50qcY2IiYIo7qDvI0ABXcHjnP1YZPw1QuV9TmJNiOHYVxiIWdtX6LB2OuL1-VZqtk7pFsSC72xcDcquZYkICSrXW5nrN7q4BI0ilbEgCNYf6HTGmLMmLBOfFqMz5v8IX0ElZVU0xlC9fquFSIa3LfokIhEg",
    "results": [
        {
            "geometry": {
                "location": {
                    "lat": -42.781144,
                    "lng": 147.0622665
                },
                "viewport": {
                    "northeast": {
                        "lat": -42.75857608597855,
                        "lng": 147.0920986679486
                    },
                    "southwest": {
                        "lat": -42.8056163856086,
                        "lng": 147.019694690057

In [21]:
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,Mirny,RU,62.5353,113.9611,Mirny
1,Geraldton,AU,-28.7667,114.6000,Geraldton
2,Cabo San Lucas,MX,22.8909,-109.9124,Cabo San Lucas
3,Cayenne,GF,4.9333,-52.3333,Cayenne
4,Nang Rong,TH,14.6377,102.7914,Nang Rong


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [24]:
# Add marker layer ontop of heat map
markers = hotel_df[["Lat", "Lng"]]
fig.gmaps.figure()
# Display figure
fig

AttributeError: 'Figure' object has no attribute 'gmaps'